In [80]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader #mini_batch를 이용해서 큰 데이터를 작은데이터를 나눠서 계산할 준비
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter
from sklearn.linear_model import LinearRegression

plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [81]:
# CustomDataset은 Dataset을 상속하고 
class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor): #__: 매직메서드
        self.x = x_tensor
        self.y = y_tensor
    
    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x) #batch_size로 나눌때 적당히 나누기 위해서

In [82]:
# 정답지
true_w = 2
true_b = 1
N = 100
np.random.seed(42)
x = np.random.rand(N, 1)
epsilon = 0.1 * np.random.randn(N, 1)   # ❗약간의 noise추가
y = true_b + ( true_w * x) + epsilon

# 8:2로 나눔
idx = np.arange(N)
np.random.shuffle(idx)
train_idx = idx[: int(N * 0.8)]
val_idx = idx[int(N * 0.8): ]
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# 모델
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

dataset = CustomDataset(x_train_tensor, y_train_tensor)

ratio = .8
n_total = len(dataset)
n_train = int(n_total * ratio)
n_val = n_total - n_train

train_data, val_data = random_split(dataset, [n_train, n_val])
train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=16)

In [83]:
lr = 0.1
torch.manual_seed(42)
model = nn.Sequential(nn.Linear(1,1)).to(device)
optimizer = optim.SGD(model.parameters(), lr=lr)
loss_fn = nn.MSELoss(reduction="mean")

#### 학습과 검증은 따로 간다

In [84]:
# 학습용
def make_train_step_fn(model, loss_fn, optimizer):
    def perform_train_step_fn(x,y):
            model.train()
            yhat = model(x_train_tensor)
            loss = loss_fn(yhat, y_train_tensor)
            loss.backward() 
            optimizer.step()
            optimizer.zero_grad()
            return loss.item()
    return perform_train_step_fn

In [85]:
# 검증용
def make_val_step_fn(model, loss_fn):
    def perform_train_step_fn(x,y):
            model.eval()
            yhat = model(x)
            loss = loss_fn(yhat, y)
            return loss.item()
    return perform_train_step_fn

In [86]:
# mini_batch 사이즈(하이퍼파라미터: 내가 정하는)만큼 돌림 => mini_batch_loss값을 모음
def mini_batch(device, data_loader, step_fn):
    mini_batch_losses =[]
    for x_batch, y_batch in data_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        mini_batch_loss = step_fn(x_batch, y_batch)
        mini_batch_losses.append(mini_batch_loss)
    loss = np.mean(mini_batch_losses)
    return loss

++ 많이 돌린다고 좋은게 아님, 오차를 줄이는게 목적이다---!

In [87]:
n_epochs = 1000
losses = [] # ❗관리대상(오차를 줄이는게 목적이니까)
train_step_fn = make_train_step_fn(model, loss_fn, optimizer)
for epoch in range(n_epochs):
   loss = mini_batch(device, train_loader, train_step_fn)
   losses.append(loss)
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0235]))])


In [88]:
n_epochs = 200
losses = [] 
val_losses = []

train_step_fn = make_train_step_fn(model, loss_fn, optimizer)
val_step_fn = make_val_step_fn(model, loss_fn)

write = SummaryWriter("runs/simple_linear_regression")
x_sample, y_sample = next(iter(train_loader))
write.add_graph(model, x_sample.to(device))

for epoch in range(n_epochs):
   # loss = mini_batch(device, train_loader, train_step_fn)
   loss = mini_batch(device, train_loader, train_step_fn)
   losses.append(loss)
   with torch.no_grad():
      val_loss = mini_batch(device, val_loader, val_step_fn)
      val_losses.append(val_loss)
   write.add_scalars(
      main_tag="loss",
      tag_scalar_dict={"training": loss, "validation": val_loss},
      global_step=epoch
   )
write.close()
print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]])), ('0.bias', tensor([1.0235]))])


In [89]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 232096), started 0:01:09 ago. (Use '!kill 232096' to kill it.)

- 상속::
    - 상위클래스의 기능을 하위클래스도 가지도록 ``강제``
- 인터페이스::
    - 계약 관계
    - 타입안정성을 확보하기 위해서
    - 설계관측자적 측면에서 필요

- ```파이썬은 인터페이스가 없음.```
- 파이썬은 상속 안좋함 -> 내가 도대체 뭘 만들어줘야 할까?

In [90]:
next(iter(train_loader))

[tensor([[0.1960],
         [0.7608],
         [0.4722],
         [0.0254],
         [0.0055],
         [0.7132],
         [0.2713],
         [0.3664],
         [0.6075],
         [0.0740],
         [0.9219],
         [0.0636],
         [0.1849],
         [0.7290],
         [0.5613],
         [0.0885]]),
 tensor([[1.4393],
         [2.4970],
         [1.9857],
         [1.0785],
         [1.0632],
         [2.6162],
         [1.5105],
         [1.7093],
         [2.4037],
         [1.1713],
         [2.8506],
         [1.1928],
         [1.5888],
         [2.4927],
         [2.0472],
         [1.0708]])]